PyGame Snake

In [1]:
import math
import random as rnd
import pygame


pygame 2.5.2 (SDL 2.28.3, Python 3.9.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class Agent:
    def __init__(self):
        pass
    def tick(self):
        return rnd.choice([pygame.K_w,pygame.K_a,pygame.K_s,pygame.K_d])


In [1]:
agt = Agent()

[1, 2]


In [15]:
#https://www.pygame.org/docs/
# Example file showing a circle moving on screen


# pygame setup
pygame.init()
screen = pygame.display.set_mode((1280, 720))
clock = pygame.time.Clock()
running = True
dt = 0

player_pos = pygame.Vector2(screen.get_width() / 2, screen.get_height() / 2)
tails=[]
tail_length =1
player_dest_y = 100
player_dest_x = 0
ct=0
random_obj = pygame.Vector2(screen.get_width() / 4, screen.get_height() / 4)
while running:
    
    # poll for events
    # pygame.QUIT event means the user clicked X to close your window
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
    
    dst = math.sqrt( (player_pos.y-random_obj.y) ** 2 + (player_pos.x-random_obj.x) ** 2 )
    if dst < 80:
        rnd.randint(0,screen.get_width())
        random_obj = pygame.Vector2(rnd.randint(0,screen.get_width()), rnd.randint(0,screen.get_height()))   
        tail_length+=1
        print(tail_length)
    if (screen.get_width() <= player_pos.x or screen.get_height() <= player_pos.y 
        or  player_pos.x <=0 or player_pos.y <=0) :
        pygame.quit()
        
    screen.fill("purple")

    pygame.draw.circle(screen, "red", player_pos, 40)
    for t in tails:
        pygame.draw.circle(screen, "red", t, 40)
    pygame.draw.circle(screen, "blue", random_obj, 40)

    keys = pygame.key.get_pressed()
    if keys[pygame.K_w]:
        player_dest_y,player_dest_x = -100,0
    elif keys[pygame.K_s]:
        player_dest_y,player_dest_x = 100,0
    elif keys[pygame.K_a]:
        player_dest_y,player_dest_x = 0,-100
    elif keys[pygame.K_d]:
        player_dest_y,player_dest_x = 0,100
    ct+=1
    if ct%30 == 0:
        tails.append(player_pos.copy())
        ct=0

    if len(tails) > tail_length:
        tails = tails[-tail_length:]
    player_pos.y += player_dest_y* dt
    player_pos.x += player_dest_x * dt    
  
    pygame.display.flip()

    # limits FPS to 60
    # dt is delta time in seconds since last frame, used for framerate-
    # independent physics.
    dt = clock.tick(60) /1000

pygame.quit()

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
